In [ ]:
!nvidia-smi
import torch
print(torch.cuda.is_available())  # Should print: True
print(torch.cuda.get_device_name(0))  # Should print: Tesla T4
!pip install torch torchvision torchaudio transformers rasterio geopandas streamlit pyngrok pandas numpy matplotlib scikit-learn opencv-python
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Cell 1: Import Libraries
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import cv2
from transformers import SegformerForSemanticSegmentation
from sklearn.metrics import accuracy_score, jaccard_score
import matplotlib.pyplot as plt
import rasterio
import geopandas as gpd
from shapely.geometry import Polygon
import pandas as pd
import torchvision.transforms as transforms
from google.colab import drive

In [ ]:
# Cell 2: Define Dataset Classes
class KaggleWildfireDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = ['nowildfire', 'wildfire']
        self.images = []
        self.labels = []
        for cls_idx, cls in enumerate(self.classes):
            cls_dir = os.path.join(root_dir, cls)
            if os.path.exists(cls_dir):
                for img_name in os.listdir(cls_dir):
                    img_path = os.path.join(cls_dir, img_name)
                    try:
                        with Image.open(img_path) as img:
                            img.load()
                            self.images.append(img_path)
                            self.labels.append(cls_idx)
                    except Exception as e:
                        print(f"Skipping corrupted file: {img_path} - {e}")
            else:
                print(f"Directory not found: {cls_dir}")

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')
        image = np.array(image)
        if self.transform:
            image = self.transform(image)
        return image, label

class EuroSATDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = sorted(os.listdir(root_dir))
        self.images = []
        self.labels = []
        for cls_idx, cls in enumerate(self.classes):
            cls_dir = os.path.join(root_dir, cls)
            if os.path.isdir(cls_dir):
                for img_name in os.listdir(cls_dir):
                    self.images.append(os.path.join(cls_dir, img_name))
                    self.labels.append(cls_idx)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')
        image = np.array(image)
        if self.transform:
            image = self.transform(image)
        return image, label

class DeepGlobeDataset(Dataset):
    def __init__(self, metadata_file, class_dict_file, root_dir, transform=None, split='train'):
        self.root_dir = root_dir
        self.transform = transform
        self.split = split
        # Load metadata
        self.metadata = pd.read_csv(metadata_file)
        self.metadata = self.metadata[self.metadata['split'] == split]
        self.image_paths = self.metadata['sat_image_path'].apply(lambda x: os.path.join(root_dir, x)).tolist()
        self.mask_paths = self.metadata['mask_path'].apply(lambda x: os.path.join(root_dir, x) if pd.notna(x) else None).tolist()
        # Load class dictionary
        self.class_dict = pd.read_csv(class_dict_file)
        self.class_to_idx = {row['name']: idx for idx, row in self.class_dict.iterrows()}
        self.color_to_class = {tuple(row[['r', 'g', 'b']].values): idx for idx, row in self.class_dict.iterrows()}

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        image = np.array(image)

        if self.mask_paths[idx] is not None and os.path.exists(self.mask_paths[idx]):
            mask = Image.open(self.mask_paths[idx])
            mask = np.array(mask)
            # Convert RGB mask to class indices
            mask_converted = np.zeros((mask.shape[0], mask.shape[1]), dtype=np.uint8)
            for color, class_idx in self.color_to_class.items():
                mask_converted[np.all(mask == color, axis=2)] = class_idx
            # Resize mask to match image
            mask_image = Image.fromarray(mask_converted)
            mask_image = mask_image.resize((512, 512), Image.NEAREST)  # Use NEAREST for segmentation masks
            mask_converted = np.array(mask_image)
        else:
            mask_converted = np.zeros((512, 512), dtype=np.uint8)  # Match image size

        if self.transform:
            image = self.transform(image)
            mask_converted = torch.tensor(mask_converted, dtype=torch.long)

        return image, mask_converted

In [ ]:
# Cell 3: Define MSSM-SCDNet Model
import torch
import torch.nn as nn
from transformers import SegformerForSemanticSegmentation

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

class MSSM_SCDNet(nn.Module):
    def __init__(self, num_classes=7, freeze_backbone=False):  # 7 classes from DeepGlobe
        super(MSSM_SCDNet, self).__init__()
        self.segformer = SegformerForSemanticSegmentation.from_pretrained(
            'nvidia/segformer-b0-finetuned-ade-512-512',
            num_labels=num_classes,
            ignore_mismatched_sizes=True
        )
        if freeze_backbone:
            for param in self.segformer.parameters():
                param.requires_grad = False
        self.cam = nn.Conv2d(num_classes, num_classes, kernel_size=1)
        self.sam = nn.Sequential(
            nn.Conv2d(num_classes, num_classes // 2, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(num_classes // 2, num_classes, kernel_size=3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        if x.shape[1] != 3:
            raise ValueError(f"Expected 3 input channels, got {x.shape[1]}")
        outputs = self.segformer(x)
        logits = outputs.logits
        logits = nn.functional.interpolate(logits, size=x.shape[2:], mode='bilinear', align_corners=False)
        cam_out = self.cam(logits)
        sam_out = self.sam(logits)
        logits = logits * sam_out
        return logits

# Initialize model (unfrozen for full retraining)
segmenter = MSSM_SCDNet(num_classes=7, freeze_backbone=False).to(device)
print("Model initialized with 7 classes, full retraining recommended.")

In [ ]:
# Cell 4: Define Classification Model (Kaggle Wildfire)
class WildfireClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super(WildfireClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 87 * 87, 256)  # Adjusted for 350x350 resize
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Cell 5: Define Transforms
deepglobe_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

kaggle_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((350, 350)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

eurosat_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# Cell 6: Load Datasets
# Mount Google Drive
#drive.mount('/content/drive')

data_root = '/content/drive/MyDrive/WildfireProject/datasets/'

# Kaggle Wildfire
kaggle_train = KaggleWildfireDataset(os.path.join(data_root, 'wildfire_prediction/train'), transform=kaggle_transform)
kaggle_val = KaggleWildfireDataset(os.path.join(data_root, 'wildfire_prediction/valid'), transform=kaggle_transform)
kaggle_test = KaggleWildfireDataset(os.path.join(data_root, 'wildfire_prediction/test'), transform=kaggle_transform)
kaggle_train_loader = DataLoader(kaggle_train, batch_size=4, shuffle=True)
kaggle_val_loader = DataLoader(kaggle_val, batch_size=4, shuffle=False)
kaggle_test_loader = DataLoader(kaggle_test, batch_size=4, shuffle=False)

# EuroSAT
eurosat_root = os.path.join(data_root, 'eurosat')
eurosat_dataset = EuroSATDataset(eurosat_root, transform=eurosat_transform)
train_size = int(0.7 * len(eurosat_dataset))
val_size = int(0.15 * len(eurosat_dataset))
test_size = len(eurosat_dataset) - train_size - val_size
eurosat_train, eurosat_val, eurosat_test = torch.utils.data.random_split(
    eurosat_dataset, [train_size, val_size, test_size]
)
eurosat_train_loader = DataLoader(eurosat_train, batch_size=4, shuffle=True)
eurosat_val_loader = DataLoader(eurosat_val, batch_size=4, shuffle=False)
eurosat_test_loader = DataLoader(eurosat_test, batch_size=4, shuffle=False)

# DeepGlobe
deepglobe_train = DeepGlobeDataset(
    metadata_file=os.path.join(data_root, 'deepglobe/metadata.csv'),
    class_dict_file=os.path.join(data_root, 'deepglobe/class_dict.csv'),
    root_dir=os.path.join(data_root, 'deepglobe'),
    transform=deepglobe_transform,
    split='train'
)
deepglobe_val = DeepGlobeDataset(
    metadata_file=os.path.join(data_root, 'deepglobe/metadata.csv'),
    class_dict_file=os.path.join(data_root, 'deepglobe/class_dict.csv'),
    root_dir=os.path.join(data_root, 'deepglobe'),
    transform=deepglobe_transform,
    split='valid'
)
deepglobe_test = DeepGlobeDataset(
    metadata_file=os.path.join(data_root, 'deepglobe/metadata.csv'),
    class_dict_file=os.path.join(data_root, 'deepglobe/class_dict.csv'),
    root_dir=os.path.join(data_root, 'deepglobe'),
    transform=deepglobe_transform,
    split='test'
)
deepglobe_train_loader = DataLoader(deepglobe_train, batch_size=2, shuffle=True)  # Increased to 2
deepglobe_val_loader = DataLoader(deepglobe_val, batch_size=2, shuffle=False)
deepglobe_test_loader = DataLoader(deepglobe_test, batch_size=2, shuffle=False)

# Verify dataset sizes
print(f"DeepGlobe Train: {len(deepglobe_train)}, Val: {len(deepglobe_val)}, Test: {len(deepglobe_test)}")
print(f"EuroSAT Train: {len(eurosat_train)}, Val: {len(eurosat_val)}, Test: {len(eurosat_test)}")
print(f"Kaggle Train: {len(kaggle_train)}, Val: {len(kaggle_val)}, Test: {len(kaggle_test)}")

In [ ]:
# Cell 7: Initialize Models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Verify GPU
!nvidia-smi

# Wildfire Classifier
classifier = WildfireClassifier(num_classes=2).to(device)
classifier_criterion = nn.CrossEntropyLoss()
classifier_optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)

# Segmentation Model
segmenter = MSSM_SCDNet(num_classes=7).to(device)
segmenter_criterion = nn.CrossEntropyLoss()
segmenter_optimizer = torch.optim.Adam(segmenter.parameters(), lr=0.001)

In [ ]:
# Cell 8: Train Model
import torch
import torch.nn.functional as F
import os

# Training setup (using segmenter from Cell 7 and loader from Cell 6)
optimizer = torch.optim.Adam(segmenter.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=255)  # Handle potential ignore index in masks

num_epochs = 5  # 5 epochs as requested
segmenter.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (images, masks) in enumerate(deepglobe_train_loader):  # Use your existing loader
        images, masks = images.to(device), masks.to(device)
        optimizer.zero_grad()
        outputs = segmenter(images)
        # Resize masks to match output (assuming 512x512 from transform)
        masks = F.interpolate(masks.unsqueeze(1).float(), size=(512, 512), mode='nearest').squeeze(1).long()
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 10 == 9:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}], Loss: {running_loss/10:.4f}")
            running_loss = 0.0
    print(f"Epoch [{epoch+1}/{num_epochs}] completed, Average Loss: {running_loss/len(deepglobe_train_loader):.4f}")

# Save model
os.makedirs('/content/drive/MyDrive/WildfireProject/models', exist_ok=True)
torch.save(segmenter.state_dict(), '/content/drive/MyDrive/WildfireProject/models/segmenter_retrained.pth')
print("Model saved to /content/drive/MyDrive/WildfireProject/models/segmenter_retrained.pth")

In [ ]:
# Cell 9: Train Segmenter (DeepGlobe)
def train_segmenter(model, train_loader, val_loader, criterion, optimizer, num_epochs=2):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for images, masks in train_loader:
            images = images.to(device)
            masks = masks.to(device)
            outputs = model(images)
            loss = criterion(outputs, masks)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        print(f'Segmenter Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader)}')

        model.eval()
        val_loss = 0
        with torch.no_grad():
            for images, masks in val_loader:
                images = images.to(device)
                masks = masks.to(device)
                outputs = model(images)
                val_loss += criterion(outputs, masks).item()
        print(f'Segmenter Epoch {epoch+1}, Val Loss: {val_loss/len(val_loader)}')

print("Training Segmenter (DeepGlobe)...")
train_segmenter(segmenter, deepglobe_train_loader, deepglobe_val_loader, segmenter_criterion, segmenter_optimizer, num_epochs=2)

In [ ]:
# Cell 10: Evaluate Models
def evaluate_classifier(model, test_loader):
    model.eval()
    predictions = []
    ground_truths = []
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)
            predictions.extend(preds.cpu().numpy())
            ground_truths.extend(labels.cpu().numpy())
    accuracy = accuracy_score(ground_truths, predictions)
    print(f'Classifier Test Accuracy: {accuracy}')
    return predictions, ground_truths

def evaluate_segmenter(model, test_loader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for images, masks in test_loader:
            images = images.to(device)
            outputs = model(images)
            preds = torch.argmax(outputs, dim=1)
            predictions.append(preds.cpu().numpy())
    print('Segmenter Test Inference Done (No mIoU due to missing test masks)')
    return predictions

print("Evaluating Classifier...")
evaluate_classifier(classifier, kaggle_test_loader)

print("Evaluating Segmenter...")
evaluate_segmenter(segmenter, deepglobe_test_loader)

In [ ]:
# Cell 11: Generate Change Map
import torch
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import os

def generate_change_map(segmenter, image_path):  # Removed classifier dependency
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Load and preprocess image
    transform = transforms.Compose([
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)
    print(f"Input shape: {image_tensor.shape}")

    # Segmenter inference (skip classifier for now)
    segmenter.eval()
    with torch.no_grad():
        seg_output = segmenter(image_tensor)
        print(f"Segmenter output shape: {seg_output.shape}")
        seg_pred = torch.argmax(seg_output, dim=1).squeeze().cpu().numpy()

    # Save change map
    output_path = '/content/drive/MyDrive/WildfireProject/outputs/change_map.png'
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    Image.fromarray((seg_pred * 255 / 6).astype(np.uint8)).save(output_path)
    print(f"Change map saved to {output_path}")

# Load retrained segmenter
segmenter = MSSM_SCDNet(num_classes=7).to(device)
segmenter.load_state_dict(torch.load('/content/drive/MyDrive/WildfireProject/models/segmenter_retrained.pth'))
segmenter.eval()

# Run
test_image_path = os.path.join(data_root, 'deepglobe/test/100877_sat.jpg')
generate_change_map(segmenter, test_image_path)

In [ ]:
# Cell 12: Compute NDVI (Placeholder)
def compute_ndvi(image_path):
    print("NDVI requires multi-spectral bands (NIR, Red). EuroSAT is RGB-only.")
    # Future: Use Sentinel-2 data for NDVI
    return None

In [ ]:
# Cell 13: Save Models
os.makedirs('/content/drive/MyDrive/WildfireProject/models', exist_ok=True)
torch.save(classifier.state_dict(), '/content/drive/MyDrive/WildfireProject/models/classifier.pth')
torch.save(segmenter.state_dict(), '/content/drive/MyDrive/WildfireProject/models/segmenter_retrained.pth')
print("Models saved to /content/drive/MyDrive/WildfireProject/models/")

In [ ]:
# Define paths
base_dir = "/content/drive/MyDrive/WildfireProject/datasets/deepglobe/"
model_path = "/content/drive/MyDrive/WildfireProject/models/segmenter_retrained.pth"
metadata_csv = "/content/drive/MyDrive/WildfireProject/datasets/deepglobe/metadata.csv"
class_dict_csv = "/content/drive/MyDrive/WildfireProject/datasets/deepglobe/class_dict.csv"

# Define the model (same as in your app)
class MSSM_SCDNet(torch.nn.Module):
    def __init__(self, num_classes=7):
        super(MSSM_SCDNet, self).__init__()
        from transformers import SegformerForSemanticSegmentation
        self.segformer = SegformerForSemanticSegmentation.from_pretrained(
            'nvidia/segformer-b0-finetuned-ade-512-512',
            num_labels=num_classes,
            ignore_mismatched_sizes=True
        )
        self.cam = torch.nn.Conv2d(num_classes, num_classes, kernel_size=1)
        self.sam = torch.nn.Sequential(
            torch.nn.Conv2d(num_classes, num_classes // 2, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(num_classes // 2, num_classes, kernel_size=3, padding=1),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        outputs = self.segformer(x)
        logits = outputs.logits
        logits = torch.nn.functional.interpolate(logits, size=x.shape[2:], mode='bilinear', align_corners=False)
        cam_out = self.cam(logits)
        sam_out = self.sam(logits)
        logits = logits * sam_out
        return logits

# Load model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
segmenter = MSSM_SCDNet(num_classes=7).to(device)
segmenter.load_state_dict(torch.load(model_path, map_location=device))
segmenter.eval()

# Image preprocessing
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load metadata CSV and filter for train split
metadata = pd.read_csv(metadata_csv)
train_metadata = metadata[metadata['split'] == 'train']

# Filter for rows with valid string mask paths
train_metadata = train_metadata[train_metadata['mask_path'].apply(lambda x: isinstance(x, str))]
image_paths = train_metadata['sat_image_path'].apply(lambda x: os.path.join(base_dir, x)).tolist()
mask_paths = train_metadata['mask_path'].apply(lambda x: os.path.join(base_dir, x)).tolist()

# Load class_dict.csv for class mapping
class_dict = pd.read_csv(class_dict_csv)
class_rgb = {i: (r, g, b) for i, (name, r, g, b) in enumerate(class_dict.values)}  # Map index to RGB
class_names = class_dict['name'].tolist()

def rgb_to_class(mask):
    """Convert RGB mask to class indices based on class_dict.csv."""
    mask = np.array(mask.convert('RGB'))  # Ensure RGB mode
    h, w, _ = mask.shape
    class_map = np.zeros((h, w), dtype=np.uint8)
    for class_idx, (r, g, b) in class_rgb.items():
        class_map[(mask[:, :, 0] == r) & (mask[:, :, 1] == g) & (mask[:, :, 2] == b)] = class_idx
    return class_map

# Collect predictions and ground truths
all_preds = []
all_labels = []

for img_path, mask_path in zip(image_paths[:10], mask_paths[:10]):  # Limit to 10 for speed
    # Load image
    image = Image.open(img_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)

    # Predict
    with torch.no_grad():
        seg_output = segmenter(image_tensor)
        pred = torch.argmax(seg_output, dim=1).squeeze().cpu().numpy()
        pred = np.clip(pred, 0, 6)  # Ensure predictions are within 0-6

    # Load and convert ground truth mask
    try:
        mask = Image.open(mask_path)
        mask = rgb_to_class(mask)  # Convert RGB to class indices
        # Convert mask to PIL Image for resizing
        mask_pil = Image.fromarray(mask)
        mask_resized = mask_pil.resize((512, 512), Image.NEAREST)  # Resize using PIL
        mask = np.array(mask_resized)  # Convert back to NumPy
        mask = np.clip(mask, 0, 6)  # Ensure masks are within 0-6
    except FileNotFoundError:
        print(f"Mask not found at {mask_path}, skipping...")
        continue

    all_preds.append(pred)
    all_labels.append(mask)

all_preds = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)

# Debugging output
print("Shape of all_preds:", all_preds.shape)
print("Unique values in all_preds:", np.unique(all_preds))
print("Shape of all_labels:", all_labels.shape)
print("Unique values in all_labels:", np.unique(all_labels))

# Calculate metrics
num_classes = 7
iou_per_class = np.zeros(num_classes)
for c in range(num_classes):
    true_pos = np.sum((all_preds == c) & (all_labels == c))
    false_pos = np.sum((all_preds == c) & (all_labels != c))
    false_neg = np.sum((all_preds != c) & (all_labels == c))
    iou = true_pos / (true_pos + false_pos + false_neg) if (true_pos + false_pos + false_neg) > 0 else 0
    iou_per_class[c] = iou

mIoU = np.mean(iou_per_class)

accuracy = np.mean(all_preds == all_labels)
precision = precision_score(all_labels.flatten(), all_preds.flatten(), average='weighted', zero_division=0)
recall = recall_score(all_labels.flatten(), all_preds.flatten(), average='weighted', zero_division=0)
f1 = f1_score(all_labels.flatten(), all_preds.flatten(), average='weighted', zero_division=0)
kappa = cohen_kappa_score(all_labels.flatten(), all_preds.flatten())
cm = confusion_matrix(all_labels.flatten(), all_preds.flatten())
oa = accuracy  # Overall Accuracy is the same as accuracy for pixel-wise classification

# Print metrics for reference
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")
print(f"mIoU: {mIoU:.2f}")
print(f"Cohen's Kappa: {kappa:.2f}")
print(f"Overall Accuracy (OA): {oa:.2f}")

In [ ]:
# Bar Chart for Accuracy, Precision, Recall, F1-score
plt.figure(figsize=(8, 6))
metrics = [accuracy, precision, recall, f1]
metric_names = ['Accuracy', 'Precision', 'Recall', 'F1-score']
plt.bar(metric_names, metrics, color=['blue', 'orange', 'green', 'red'])
plt.title('Classification Metrics (Training Subset)')
plt.ylim(0, 1)
for i, v in enumerate(metrics):
    plt.text(i, v + 0.02, f'{v:.2f}', ha='center')
plt.ylabel('Score')
plt.savefig('/content/drive/MyDrive/WildfireProject/outputs/classification_metrics.png')
plt.show()

# Bar Chart for Accuracy, Precision, Recall, F1-score
plt.figure(figsize=(8, 6))
metrics = [accuracy, precision, recall, f1]
metric_names = ['Accuracy', 'Precision', 'Recall', 'F1-score']
plt.bar(metric_names, metrics, color=['blue', 'orange', 'green', 'red'])
plt.title('Classification Metrics (Training Subset)')
plt.ylim(0, 1)
for i, v in enumerate(metrics):
    plt.text(i, v + 0.02, f'{v:.2f}', ha='center')
plt.ylabel('Score')
plt.savefig('/content/drive/MyDrive/WildfireProject/outputs/classification_metrics.png')
plt.show()

# Confusion Matrix Heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix (Training Subset)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.savefig('/content/drive/MyDrive/WildfireProject/outputs/confusion_matrix.png')
plt.show()


# Bar Chart for OA and Kappa
plt.figure(figsize=(8, 6))
metrics_oa_kappa = [oa, kappa]
metric_names_oa_kappa = ['Overall Accuracy', "Cohen's Kappa"]
plt.bar(metric_names_oa_kappa, metrics_oa_kappa, color=['cyan', 'magenta'])
plt.title('Overall Accuracy and Kappa (Training Subset)')
plt.ylim(0, 1)
for i, v in enumerate(metrics_oa_kappa):
    plt.text(i, v + 0.02, f'{v:.2f}', ha='center')
plt.ylabel('Score')
plt.savefig('/content/drive/MyDrive/WildfireProject/outputs/oa_kappa_metrics.png')
plt.show()